In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import time

import os #MAKDIR & DELETING FILES
import csv #COMMA SEPARATED VALUES LIBRARY
import requests #DOWNLOADING IMAGES

In [2]:
#TO WŁĄCZA STRONĘ
PATH = r"C:\Users\PC\1st\chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.get("https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/mazowieckie")

In [3]:
def create_folder_in_current_directory(folderName):#FOLDER NAME TO STRING - MA BYĆ W CUDZYSŁOWIE
    try:
        os.mkdir(folderName)
    except:
        print("Nie stworzono folderu o nazwie: '" + folderName+"'")
        
#DIRECTORY MA BYĆ Z UKOŚNIKAMI "/", A NIE "\"!!! POZA TYM OBA ARGUMENTY W CUDZYSŁOWACH
def create_folder_in_given_directory(directory, folderName):
    path = os.path.join(directory, folderName)
    try:
        os.mkdir(path)
    except:
        print("Nie stworzono folderu o nazwie: '" + folderName+"' w: " + path)
        
def append_list_of_links_of_images(list, object):#object holds img elements within itself
    images = object.find_elements_by_class_name("image-gallery-image")
    for i in range(len(images)):
        if(images[i].get_attribute("src") in imagesLinks):
            continue
        else:
            imagesLinks.append(images[i].get_attribute("src"))

In [4]:
#TO TWOŻY LISTĘ LINKÓW "links".
posts = driver.find_elements_by_css_selector("a[data-cy='listing-item-link']")
links = []
#print("To jest lista ogłoszeń:")
for post in posts:
    links.append(post.get_attribute("href"))
    #print(links[posts.index(post)])#To jest tylko do wypisywania linków. Można usunąć.
print("Liczba ogłoszeń: " + str(len(links)))

Liczba ogłoszeń: 27


In [ ]:
for iterator in range(len(links)):
    create_folder_in_given_directory("./scraped", str(iterator))#ZROBIĆ ITERACJĘ Z TYM ZEREM
    directory = "./scraped/" + str(iterator) + "/"#ZAMIAST ZERA TUTAJ DAĆ TEN ITERATOR STR(I)
    
    
    
    data = []#TU BĘDĄ WSZYSTKIE PARAMETRY MIESZKANIA
    driver.get(links[iterator])#WEJŚĆ NA STRONĘ
    
    
    
    cena = driver.find_element_by_css_selector("strong[aria-label='Cena']")
    cenaMetra = driver.find_element_by_css_selector("div[aria-label='Cena za metr kwadratowy']")
    title = driver.find_element_by_tag_name("h1")
    data.append("Tytuł:")
    data.append(title.text)
    data.append("Cena:")
    data.append(cena.text)
    data.append("Cena za metr kwadratowy:")
    data.append(cenaMetra.text[:-1])
    
    
    
    table = driver.find_element_by_class_name("egzohkh2")#ZNAJDUJE TABELEKĘ ZE SZCZEGÓŁAMI
    fields = table.find_elements_by_xpath(".//*")#ZNAJDUJE WSZYSTKIE ELEMENTY W TABELCE
    for field in fields:
        teksty = field.find_elements_by_xpath(".//*");#ZEBRAĆ WSZYSTKIE JEJ ELEMENTY (TYTUŁ I WARTOŚĆ, ZDJĘCIE Z JAKIEGOŚ POWODU SIE NIE ZNAJDUJE)
        if (fields.index(field)!=0):#METRY KWADRATOWE WYWALAŁY BŁĄD, WIĘC POWIERZCHNIA JEST OSOBNO
            if(len(teksty)==2):
                data.append(teksty[0].text)
                data.append(teksty[1].text)
        else:#TO JEST POWIERZCHNIA
            if(len(teksty)==2):
                data.append(teksty[0].text)#WPISUJE WARTOŚCI DO TABLICY, Z KTÓREJ PÓŹNIEJ BĘDZIE WPISYWANE DO PLIKU
                temp = teksty[1].text
                temp = temp[:-1]
                data.append(temp)
    print (data)
    
    
    
    with open(directory+'dane.csv', 'w', newline='') as csvfile:#OTWIERA DANE.CSV I USUWA WSZYSTKO CO W NICH BYŁO JEŻELI COŚ BYŁO
        pisacz = csv.writer(csvfile, delimiter=';')#POTRZEBNE DLA BIBLIOTEKI CSV
        j = 0
        i = 0
        if (len(data) > 2):
            for i in range(0, (int(len(data)/2))):
                try:
                    pisacz.writerow([data[j],data[j+1]])#PISZE PO DWIE KOMÓRKI Z DANYCH NA WIERSZ
                    j+=2
                except:
                    print("Nie udało się coś")
        else:
            print("Za mało danych")
        
        
        
    #NIE TYLKO DODATKOWE INFORMACJE MAJĄ TĄ SAMĄ KLASĘ, ALE JAKIEŚ RZECZY TYPU WYPOSARZENIE, ALBO ZABEZPIECZENIA
    infDodatkowe = driver.find_elements_by_class_name("ex3yvbv1")#LOKALIZUJE DODATKOWE INFORMACJE
    listaTekstow = ["Informacje dodatkowe:"]
    link = ["Link:"]
    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!DODAĆ ITERACJĘ NA KAŻDY LINK!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    link.append(links[0])
    for i in range(len(infDodatkowe)):
        listaTekstow.append(infDodatkowe[i].text)
    with open(directory+'dane.csv', 'a', newline='') as csvfile:
        pisacz = csv.writer(csvfile, delimiter=';')
        pisacz.writerow(listaTekstow)
        
        
        
    opis = driver.find_element_by_css_selector('div[data-cy="adPageAdDescription"]')#ZNAJDUJE OPIS, PO JEGO DATA-CY
    teksty = opis.find_elements_by_tag_name("p");#ZNAJDUJE WSZYSTKIE DZIECI OPISU
    txtf = open(directory+"desription.txt", "w")#TWORZY PLIK TEKSTOWY
    for i in range(len(teksty)):
        txtf.write(teksty[i].text + "\n")#ZAPISUJE WSZYSTKIE LINIJKI OPISU PO KOLEI 
    txtf.close()#ZAMYKA PLIK
    
    
    time.sleep(1)
    guzikNext = driver.find_element_by_css_selector("button[data-cy='ad-media-gallery.next-photo']")#ZNAJDUJE GUZIK NEXT
    try:  
        guzikCookies = driver.find_element_by_id("onetrust-accept-btn-handler")#ZNAJDUJE GUDZIK OD COOKIES (AKCEPTUJ)
        guzikCookies.click()#KLIKA TO CHOLERNE AKCEPTUJ
    except:
        print("Nie pominięto")
    time.sleep(1)
    try:
        guzikKrzyzyk = driver.find_element_by_css_selector("button[data-cy='ad.gallery.virtual-walkaround-overlay.close-button-label']")
        guzikKrzyzyk.click()
    except:
        print("Nie kliknięto krzyżyka")
    liczbaZdjec = int(driver.find_element_by_class_name("image-gallery-index-total").text)#ZNAJDUJE I ZAPISUJE LICZBĘ ZDJĘĆ
    elementZeZdjeciami = driver.find_element_by_class_name("image-gallery-slides")#NO I ZNAJDUJE ELEMENT, KTÓRY ZAWIERA WSZYSTKIE ZDJĘCIA
    imagesLinks = []
    for i in range(liczbaZdjec):#KLIKA GUZIK I WPISUJE LINKI DO TABELKI
        append_list_of_links_of_images(imagesLinks, elementZeZdjeciami)
        guzikNext.click()
        time.sleep(1)
    #print (imagesLinks)

    
    
    #print(len(imagesLinks))#LICZBA ZDJĘĆ
    create_folder_in_given_directory(directory, "zdjecia")#TWORZY FOLDER NA ZDJĘCIA
    for i in range(len(imagesLinks)):
        response = requests.get(imagesLinks[i])#NIE WIEM CO TO ROBI, ALE TE KILKA LINIJEK POBIERA ZDJĘCIE
        nazwaPliku = "photo" + str(i)+".png"#ZAZWA ZDJĘCIA
        path = os.path.join(directory, "zdjecia", nazwaPliku)#ŚCIEŻKA DO ZDJĘCIA
        if response.status_code == 200:#I TO KONKRETNIE POBIERA ZDJĘCIE. TE 3 LINIJKI \/ \/ \/
            with open(path, 'wb') as file:
                file.write(response.content)

['Tytuł:', 'Mieszkanie ul. Odkryta 65A 48,66m2 BEZPOŚREDNIO', 'Cena:', '531 000 zł', 'Cena za metr kwadratowy:', '10 912 zł/m', 'Powierzchnia:', '48,66 m', 'Obsługa zdalna:', 'Tak', 'Liczba pokoi:', '2', 'Rynek:', 'wtórny', 'Rodzaj zabudowy:', 'blok', 'Piętro:', '4', 'Liczba pięter:', '4', 'Materiał budynku:', 'żelbet', 'Okna:', 'plastikowe', 'Ogrzewanie:', 'gazowe', 'Rok budowy:', '2010', 'Stan wykończenia:', 'do zamieszkania', 'Czynsz:', '520 zł', 'Forma własności:', 'pełna własność']
Nie kliknięto krzyżyka
['Tytuł:', 'Blisko metra, po remoncie, tylko bezpośrednio', 'Cena:', '598 000 zł', 'Cena za metr kwadratowy:', '11 500 zł/m', 'Powierzchnia:', '52 m', 'Liczba pokoi:', '3', 'Rynek:', 'wtórny', 'Rodzaj zabudowy:', 'blok', 'Piętro:', '4', 'Liczba pięter:', '10', 'Materiał budynku:', 'wielka płyta', 'Okna:', 'plastikowe', 'Ogrzewanie:', 'miejskie', 'Rok budowy:', '1978', 'Forma własności:', 'spółdzielcze wł. z KW']
Nie pominięto
Nie kliknięto krzyżyka
['Tytuł:', 'Bezpośrednio Puławsk